Question 1: What is the correlation between Google Search Trends (Could also include Bing Keyword Research Tool) and Volume for stocks?

In [3]:
import pandas as pd
import plotly.express as px 
from scipy.stats import pearsonr, chi2_contingency


In [4]:
class Date: 
    def __init__(self, month: int, day: int, year: int):
        self.month = month
        self.day = day
        self.year = year
    def __str__(self) -> str:
        return f'{self.year}-{self.month}-{self.day}'
    def decrement_day(self):
        if self.day > 1:
            self.day -= 1
        else:
            # Handle days in months with 30 days or February
            if self.month in [1, 3, 5, 7, 8, 10, 12]:
                if self.month == 1:
                    self.month = 12
                    self.year -= 1
                else:
                    self.month -= 1
                self.day = 31
            elif self.month == 3:  # February case
                self.month = 2
                # Check for leap year
                if (self.year % 4 == 0 and self.year % 100 != 0) or (self.year % 400 == 0):
                    self.day = 29
                else:
                    self.day = 28
            else:  # Months with 30 days
                self.month -= 1
                self.day = 30

In [10]:
def change_dates(stock_fin):
    new_dates = []
    for index, row in stock_fin.iterrows():
        date = row['Date']
        date = date.split('-')
        year = int(date[0])
        month = int(date[1])
        day = int(date[2])
        temp = Date(month, day, year)
        temp.decrement_day()
        new_dates.append(str(temp))
    stock_fin['Date'] = new_dates    

def stock_correlation(fin_csv: str, trends_csv: str, scale:int, plot_title:str):
    trends = pd.read_csv(trends_csv, skiprows=2)
    stock_price = pd.read_csv(fin_csv)

    # prior to merge, must convert one of the data sets to the same as the other dataset
    change_dates(stock_price)

    merged_data = trends.merge(stock_price, left_on='Week', right_on='Date')
    # scale volume down by 1M
    merged_data['Volume'] = merged_data['Volume']/scale

    stock_name = fin_csv.split('.')[0]
    #fig = px.scatter(merged_data, x='Week',y=['Interest', 'Volume'], title=stock_name)
    fig = px.scatter(merged_data, x='Interest', y='Volume', title=plot_title, trendline="ols")
    fig.show()
    correlation_coefficient, p_value = pearsonr(merged_data['Interest'], merged_data['Volume'])
    print("Pearson correlation coefficient:", correlation_coefficient)
    print("P-value:", p_value)
    return merged_data




In [11]:
nvidia_data = stock_correlation('NVDA_5yr.csv', 'nvda_trends_5yr.csv', 1, "Nvidia's Google Search Trend Interest and Trading Volume")

Pearson correlation coefficient: 0.3525286613472377
P-value: 0.016267058199503328


In [12]:
tsla_data = stock_correlation('TSLA_5yr.csv', 'tsla_trends_5yr.csv', 1, "Tesla's Google Search Trend Interest and Trading Volume")

Pearson correlation coefficient: -0.042482538305072315
P-value: 0.7792261576856545


In [13]:
apple_data = stock_correlation('AAPL_5yr.csv', 'aapl_trends_5yr.csv', 1, "Apple's Google Search Trend Interest and Trading Volume")

Pearson correlation coefficient: -0.30053690424113305
P-value: 0.04241478176070451
